In [1]:
import pandas as pd
import seaborn as sns
import numpy as np 

from matplotlib import pyplot as plt
from os import path

In [2]:
env_measurements_df = pd.read_csv('../data/bahram_env_vars_raw.csv')
env_measurements_df = env_measurements_df.dropna().set_index('sample_id')
env_measurements_df.head()

,pH,Ca,Mg,P,K,N,C,d15N,d13C,C/N,DistEqu,Moisture,MAP,MAT,PET,Fire,NPP,SoilC_from_database
sample_id,,,,,,,,,,,,,,,,,,
G2761a,3.69,2.0801,1.4729,0.9072,1.9921,0.1885,2.4948,4.2231,-29.6251,13.24,3.2167,2.1311,3.3840,25.8750,2.1405,2.0043,1.039,5.022
AV103,2.43,2.0766,1.4256,1.2147,1.9529,1.5841,29.2100,3.3900,-30.5500,18.44,0.3919,2.0928,3.4804,26.5417,2.0605,2.0043,0.927,5.596
AV104,2.51,2.2700,2.1358,1.7251,2.5075,0.7061,16.1900,-0.4500,-30.4500,22.93,0.3466,2.0928,3.4794,26.5000,2.0605,2.0043,0.927,5.596
AV105,2.65,2.1991,2.2401,1.8166,2.8859,1.7613,33.6700,3.0100,-30.2800,19.12,3.5980,2.1531,3.4026,25.7167,2.0397,2.0043,0.988,6.894
AV110,2.89,2.0753,1.5848,1.1345,2.3218,0.6776,11.4400,2.1400,-30.9900,16.88,0.3806,2.0928,3.4803,26.4917,2.0605,2.0043,0.927,5.596


In [3]:
sample_metadata = pd.read_csv('../data/ENA_PRJEB19856_metadata.tsv', sep='\t')
submitted_sample_ids = [a.split('/')[-1].split('.')[0] 
                        for a,b in sample_metadata.submitted_ftp.str.split(';')]
sample_metadata['sample_id'] = submitted_sample_ids
sample_metadata.set_index('sample_id', inplace=True)
sample_metadata.head()

,study_accession,sample_accession,experiment_accession,run_accession,tax_id,scientific_name,fastq_ftp,submitted_ftp,sra_ftp
sample_id,,,,,,,,,
AV103,PRJEB19856,SAMEA103892433,ERX1933710,ERR1873185,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/005/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873185/A...,NaN
AV104,PRJEB19856,SAMEA103892434,ERX1933711,ERR1873186,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/006/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873186/A...,NaN
AV105,PRJEB19856,SAMEA103892435,ERX1933712,ERR1873187,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/007/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873187/A...,NaN
AV110,PRJEB19856,SAMEA103892436,ERX1933713,ERR1873188,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/008/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873188/A...,NaN
AV112,PRJEB19856,SAMEA103892437,ERX1933714,ERR1873189,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/009/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873189/A...,NaN


In [4]:
run_accessions = []
for i in env_measurements_df.index:
    if i in sample_metadata.index:
        run_accessions.append(sample_metadata.loc[i].run_accession)
    else:
        run_accessions.append(None)

run_accessions = pd.Series(run_accessions, env_measurements_df.index)
        
env_measurements_df['run_accession'] = run_accessions

# Manual fix for sample G2761a which is 2x in the list
env_measurements_df.at['G2761a', 'run_accession'] = 'ERR1873229'

env_measurements_df['pH_range'] = pd.cut(env_measurements_df.pH, range(14))

env_measurements_df['pH,pH_range,C,N,run_accession'.split(',')].head(5)

,pH,pH_range,C,N,run_accession
sample_id,,,,,
G2761a,3.69,"(3, 4]",2.4948,0.1885,ERR1873229
AV103,2.43,"(2, 3]",29.2100,1.5841,ERR1873185
AV104,2.51,"(2, 3]",16.1900,0.7061,ERR1873186
AV105,2.65,"(2, 3]",33.6700,1.7613,ERR1873187
AV110,2.89,"(2, 3]",11.4400,0.6776,ERR1873188


In [5]:
# rename columns
env_measurements_df.reset_index(inplace=True)
env_measurements_df.columns = ['bahram_sample_id', 'pH', 'Ca', 'Mg', 'P', 'K', 'N', 'C', 'd15N', 'd13C', 'C/N', 'DistEqu',
       'Moisture', 'MAP', 'MAT', 'PET', 'Fire', 'NPP', 'SoilC_from_database',
       'sample-id', 'pH_range']

env_measurements_df.set_index('sample-id').to_csv('../data/bahram_env_vars_for_qiime.tsv', sep='\t')
env_measurements_df.head()

,bahram_sample_id,pH,Ca,Mg,P,K,N,C,d15N,d13C,...,DistEqu,Moisture,MAP,MAT,PET,Fire,NPP,SoilC_from_database,sample-id,pH_range
0,G2761a,3.69,2.0801,1.4729,0.9072,1.9921,0.1885,2.4948,4.2231,-29.6251,...,3.2167,2.1311,3.3840,25.8750,2.1405,2.0043,1.039,5.022,ERR1873229,"(3, 4]"
1,AV103,2.43,2.0766,1.4256,1.2147,1.9529,1.5841,29.2100,3.3900,-30.5500,...,0.3919,2.0928,3.4804,26.5417,2.0605,2.0043,0.927,5.596,ERR1873185,"(2, 3]"
2,AV104,2.51,2.2700,2.1358,1.7251,2.5075,0.7061,16.1900,-0.4500,-30.4500,...,0.3466,2.0928,3.4794,26.5000,2.0605,2.0043,0.927,5.596,ERR1873186,"(2, 3]"
3,AV105,2.65,2.1991,2.2401,1.8166,2.8859,1.7613,33.6700,3.0100,-30.2800,...,3.5980,2.1531,3.4026,25.7167,2.0397,2.0043,0.988,6.894,ERR1873187,"(2, 3]"
4,AV110,2.89,2.0753,1.5848,1.1345,2.3218,0.6776,11.4400,2.1400,-30.9900,...,0.3806,2.0928,3.4803,26.4917,2.0605,2.0043,0.927,5.596,ERR1873188,"(2, 3]"
